In [38]:
import os
import numpy as np
import codecs
path_baptiste = "/home/baptiste/Documents/data/train"
path_igor = "C:/Users/Igor/Documents/Master Data Science/Big Data Analytics/Projet/Data/train"
path_sofia = "/Users/Flukmacdesof/data 2/train"



#assumes labelled data ra stored into a positive and negative folder
#returns two lists one with the text per file and another with the corresponding class 
def loadLabeled(path):

	rootdirPOS =path+'/pos'
	rootdirNEG =path+'/neg'
	data=[]
	Class=[]
	count=0
	for subdir, dirs, files in os.walk(rootdirPOS):
		
		for file in files:
			with codecs.open(rootdirPOS+"/"+file, 'r',encoding="utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc1=np.ones(len(data))
	for subdir, dirs, files in os.walk(rootdirNEG):
		
		for file in files:
			with codecs.open(rootdirNEG+"/"+file, 'r',encoding="utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc0=np.zeros(len(data)-len(tmpc1))
	Class=np.concatenate((tmpc1,tmpc0),axis=0)
	return data,Class
#loads unlabelled data	
#returns two lists
#one with the data per file and another with the respective filenames (without the file extension)
def loadUknown(path):
	rootdir=path
	data=[]
	names=[]
	for subdir, dirs, files in os.walk(rootdir):
		for file in files:
			with open(rootdir+"/"+file, 'r', endoding= "utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
				names.append(file.split(".")[0])
	return data,names

## Data loading

In [110]:
reviews, Class = loadLabeled(path_igor)

## First data cleaning (Igor):
- Remove all the HTML symbols

In [111]:
# Remove HLML signs
HTMLlist = ['<br />']

for idx, review in enumerate(reviews):
    for word in HTMLlist:
        reviews[idx] = review.replace(word,' ')

## Creating new features : Other ideas to try

- Find N-grams where it may start with a CAPITAL (As for the movie names and actor's names)


## Feature creation (Sofia):
- List punctuation (various form)
- Find smiley

In [112]:
excla = [0]*len(reviews)
inter = [0]*len(reviews)
susp = [0]*len(reviews)
for i, review in enumerate(reviews):
    for char in review:
        if char == "?":
            inter[i] += 1
        elif char == "!":
            excla[i] += 1

In [113]:
from nltk.tokenize.casual import TweetTokenizer
ttoken = TweetTokenizer(reduce_len=True)
tokenized_reviews = []

for review in reviews:
    tokenized_reviews.append(ttoken.tokenize(review))

In [114]:
for i, review in enumerate(tokenized_reviews):
    for word in review:
        if word == "...":
            susp[i] += 1

## Feature creation (Igor): 
- Length of the review
- Grad given in the review
- Movie mentionned in the review

In [44]:
# The movie list has to be completed
# - Find a list of the movie so as it matches some in the reviews : www.imdb.com
movie_list = ['Titanic']

rev_length = []
rev_word_count = []
rev_movie = []

for idr,review in enumerate(reviews):
    # Length of the review
    rev_length.append(len(review))
    
    rev_word_count.append(0)
    for word in review:
        rev_word_count[idr]+=1
        
    # Movie in the review
    for movie_name in movie_list:
        if movie_name in review:
            rev_movie.append(movie_name)
            #break
        else:
            rev_movie.append('no movie')

In [115]:
rev_grade = []
# Grade/Mark in the review
for idr,review in enumerate(reviews):
    rev_grade.append([])
    review_split= review.split(" ")

    for idw, word in enumerate(review_split):
        for idx, char in enumerate(word):
                if char == '/':
                    ten_is_there= False
                    if(idx < len(word)-2):
                        if word[idx+1] == '1' and word[idx+2] == '0':
                            ten_is_there=True
                    if(idx== len(word) -1 and idw<len(review_split)-1 and len(review_split[idw+1])>1 ):
                        if(review_split[idw+1][0]=='1' and review_split[idw+1][1]=='0'):
                            ten_is_there=True
                    if(ten_is_there):                  
                        if(idx)>0:
                            rev_grade[idr].append(word[0:idx])
                        else:
                            rev_grade[idr].append(review_split[idw-1])

In [210]:
# function for reviews with one grade
def convert_to_real_grade(grade):
    try:
        return float(grade)
    except:
        wrong = ['']
        for char in grade:
            if char in 
        return str(grade)

# functions for reviews with more than one grade
def convert_to_real_grade_2(grade):
    return str(grade[1])
    

rev_grade1 = []
for idg, grade in enumerate(rev_grade):
    if grade != []:
        if len(grade) == 1:
            rev_grade1.append(convert_to_real_grade(grade[0]))
        else:
            rev_grade1.append(-1)
            #rev_grade1.append(convert_to_real_grade_2(grade))
    else: 
        rev_grade1.append(-1)
        

In [211]:
np.unique(rev_grade1)

array(['"10', "'0", '(1', '(10', '(12', '(2', '(3', '(4', '(5.8', '(5.9',
       '(6/13', '(7', '(8', '(9', '***', '***7', '***8', '-1', '-8.0',
       '-inf', '0.0', '0.1', '0.5', '01/23', '1.0', '1.1', '1.4', '1.5',
       '1.6', '1.9', '1/2', '10.0', '11.0', '15.0', '1e+13', '1e-05',
       '2.0', '2.5', '20.0', '3.0', '3.2', '3.5', '3.75', '4.0', '4.3',
       '4.5', '4.7', '5.0', '5.2', '5.7', '500.0', '6.0', '6.3', '6.5',
       '6.6', '6.8', '7,5', '7.0', '7.4', '7.5', '7.6', '7.7', '79.0',
       '8-9', '8.0', '8.25', '8.3', '8.5', '8.75', '8.9', '84.0', '85.0',
       '9.0', '9.1', '9.4', '9.5', '9.8', '95.0', 'Frailty--8', 'Horses',
       'LOOK.(10', 'Rating-3', 'Rating-4', 'Rating-8', 'Therefore...10',
       'ZERO', '[1', 'anyway...4', 'brilliant.(10', 'fairy:4', 'fans...8',
       'film-10', 'further.9', 'in...3', 'intentions', 'living.(10',
       'me)...8', 'one...4', 'out...9', 'star', 'subject.(2',
       'this......-----3', 'time.....2', 'umm...1', 'warned...1',
    

## Second Data Cleaning (After the features creation) :
 - Punctuation
 - Stop Words


In [46]:
# Remove punctuation, lower all characters
# exclude = {',' ,'+', '<', ':', '/', ']', '(', ')', '{', '"', '_', '?', '@', '}', ...}
import string
exclude = set(string.punctuation)
for idx, review in enumerate(reviews):
    reviews[idx] = ''.join([w for w in review.lower() if w not in exclude])
    
    
# Remove stop words based on the given list - To be changed depending on the needs
from nltk.corpus import stopwords
stopwords = stopwords.words("english")
for idx, review in enumerate(reviews):
    reviews[idx] = ''.join([w for w in review if w not in stopwords])
    #problème de codage à voir

## Third Data Cleaning (Baptiste): 
- Stemmisation

In [12]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [13]:
# Steeming -> Reduce words to their initial mining

for idx, review in enumerate(reviews):
    for word in review: 
        reviews[idx] = ''.join([stemmer.stem(word)+' ' for word in review.split()])

KeyboardInterrupt: 

## Tf - Idf Matrix (Igor)

#### To be upgraded with new tf and idf functions!

In [47]:
# Features extraction with TF - IDF : get the matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

m = TfidfVectorizer()
tfidf_matrix = m.fit_transform(reviews)

print("Size of the tfidf matrix: ", tfidf_matrix.size)

('Size of the tfidf matrix: ', 2954280)


## Adding the new features to the Tf-Idf matrix
#### New features are : 
- Number of exclamation point
- Number of interrogation point
- Number of suspension point
- Review length
- Number of word (word_count)
- Movie mentionned
- Grade mentionned

- Smileys (How to deal with them?)

In [48]:
import scipy

def csr_vappend(a,b):
    """ Takes in 2 csr_matrices and appends the second one to the bottom of the first one. 
    Much faster than scipy.sparse.vstack but assumes the type to be csr and overwrites
    the first matrix instead of copying it. The data, indices, and indptr still get copied."""

    a.data = np.hstack((a.data,b.data))
    a.indices = np.hstack((a.indices,b.indices))
    a.indptr = np.hstack((a.indptr,(b.indptr + a.nnz)[1:]))
    a._shape = (a.shape[0]+b.shape[0],b.shape[1])
    return a

In [69]:
str(rev_grade[5][0])

'7.7'

In [61]:
excla1 = scipy.sparse.csr_matrix(excla)
inter1 = scipy.sparse.csr_matrix(inter)
susp1 = scipy.sparse.csr_matrix(susp)
rev_length1 = scipy.sparse.csr_matrix(rev_length)
rev_word_count1 = scipy.sparse.csr_matrix(rev_word_count)
rev_grade1 = scipy.sparse.csr_matrix(rev_grade)
rev_movie1 = scipy.sparse.csr_matrix(np.array(rev_movie))

TypeError: Cannot cast array data from dtype('O') to dtype('bool') according to the rule 'safe'

In [ ]:
csr_vappend(tfidf_matrix, excla1)
csr_vappend(tfidf_matrix, inter1)

##  CSV Creation (Baptiste):

Create 5 csv : train_train.csv, train_test.csv, y_train_train.csv, y_train_test.csv, test.csv

In [15]:
# Split the tf-idf matrix into two data sets to process the cross validation : training and test set
from sklearn.cross_validation import train_test_split

data_train, data_test, label_train, label_test = train_test_split(tfidf_matrix, Class, test_size = 0.3, random_state = 42)

In [16]:

import csv

def save_sparse_csr(filename, array):
    np.savez(filename, data = array.data, indices = array.indices, 
             indptr = array.indptr, shape = array.shape)
    
def load_sparse_csr(filename):
    loader = np.load(filename)
    return scipy.sparse.csr_matrix(( loader['data'], loader['indices'], loader['indptr']),
                     shape = loader['shape'])

def save_csv(filename, array):
    with open(filename, 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter = '\n')
        writer.writerow(array)
        
def load_csv(filename):
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter = '\n')
        array = [float(row[0]) for row in reader]
        return array

In [17]:
save_sparse_csr('data_train', data_train)
save_sparse_csr('data_test', data_test)
save_csv('label_train.csv', label_train)
save_csv('label_test.csv', label_test)

<17500x78042 sparse matrix of type '<type 'numpy.float64'>'
	with 2073379 stored elements in Compressed Sparse Row format>

<32500x78042 sparse matrix of type '<type 'numpy.float64'>'
	with 3835155 stored elements in Compressed Sparse Row format>

<40000x78042 sparse matrix of type '<type 'numpy.float64'>'
	with 4716043 stored elements in Compressed Sparse Row format>

<7500x78042 sparse matrix of type '<type 'numpy.float64'>'
	with 880888 stored elements in Compressed Sparse Row format>